In [60]:
import pandas as pd
import numpy as np
import seaborn as sns

In [61]:
df = pd.read_csv("Salary_Data.csv")
df.head()

,YearsExperience,Salary
0,1.1,39343.0
1,1.3,46205.0
2,1.5,37731.0
3,2.0,43525.0
4,2.2,39891.0


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YearsExperience  30 non-null     float64
 1   Salary           30 non-null     float64
dtypes: float64(2)
memory usage: 608.0 bytes


In [63]:
df.describe()

,YearsExperience,Salary
count,30.000000,30.000000
mean,5.313333,76003.000000
std,2.837888,27414.429785
min,1.100000,37731.000000
25%,3.200000,56720.750000
50%,4.700000,65237.000000
75%,7.700000,100544.750000
max,10.500000,122391.000000


In [64]:
x = df[['YearsExperience']]
y = df[['Salary']]

In [65]:
x = np.array(x).astype('float32')
y = np.array(y).astype('float32')

In [66]:
y

array([[ 39343.],
       [ 46205.],
       [ 37731.],
       [ 43525.],
       [ 39891.],
       [ 56642.],
       [ 60150.],
       [ 54445.],
       [ 64445.],
       [ 57189.],
       [ 63218.],
       [ 55794.],
       [ 56957.],
       [ 57081.],
       [ 61111.],
       [ 67938.],
       [ 66029.],
       [ 83088.],
       [ 81363.],
       [ 93940.],
       [ 91738.],
       [ 98273.],
       [101302.],
       [113812.],
       [109431.],
       [105582.],
       [116969.],
       [112635.],
       [122391.],
       [121872.]], dtype=float32)

In [103]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=100)

In [104]:
import sagemaker
import boto3
from sagemaker import Session
Sagemaker_session = sagemaker.Session()
bucket = Session().default_bucket()
prefix = 'linear_learner'
role = sagemaker.get_execution_role()

In [105]:
print(bucket)

sagemaker-us-east-1-839885035127


In [106]:
print(role)

arn:aws:iam::839885035127:role/sagemaker_fullrole


In [107]:
x_train.shape

(24, 1)

In [108]:
y_train.shape

(24, 1)

In [109]:
y_train = y_train[:,0]

In [110]:
y_train

array([112635., 105582.,  60150.,  83088., 101302.,  55794.,  39891.,
       121872.,  39343.,  46205.,  81363.,  61111.,  93940.,  98273.,
        37731.,  91738.,  63218.,  66029.,  67938., 113812.,  54445.,
        43525., 109431.,  64445.], dtype=float32)

In [111]:
import io
import sagemaker.amazon.common as smac
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf,x_train,y_train)
buf.seek(0)

0

In [112]:
import os
key = "linear-train-data"
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print(s3_train_data)

s3://sagemaker-us-east-1-839885035127/linear_learner/train/linear-train-data


In [113]:
y_test = y_test[:,0]

In [114]:
y_test

array([ 57189., 116969., 122391.,  57081.,  56642.,  56957.],
      dtype=float32)

In [115]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf,x_test,y_test)
buf.seek(0)

0

In [116]:
key = 'linear-test-data'
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix,'test',key)).upload_fileobj(buf)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket,prefix,key)
print(s3_test_data)

s3://sagemaker-us-east-1-839885035127/linear_learner/test/linear-test-data


In [117]:
output_location = 's3://{}/{}/output'.format(bucket,prefix)
print(output_location)

s3://sagemaker-us-east-1-839885035127/linear_learner/output


In [118]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [122]:
linear = sagemaker.estimator.Estimator(container,
          role, 
          train_instance_count = 1, 
          train_instance_type = 'ml.m5.2xlarge',
          output_path = output_location,
          sagemaker_session = Sagemaker_session)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [124]:
linear.set_hyperparameters(feature_dim = 1,
                           predictor_type = 'regressor',
                           mini_batch_size = 5,
                           epochs = 5,
                           num_models = 5,
                           loss = 'absolute_loss')
linear.fit({'train': s3_train_data})

2022-12-07 13:01:43 Starting - Starting the training job...
2022-12-07 13:02:08 Starting - Preparing the instances for trainingProfilerReport-1670418103: InProgress
......
2022-12-07 13:03:11 Downloading - Downloading input data...
2022-12-07 13:03:32 Training - Downloading the training image......
2022-12-07 13:04:37 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/07/2022 13:04:40 INFO 139878988891968] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias':

In [125]:
linear_regressor = linear.deploy(initial_instance_count = 1,
                                 instance_type = 'ml.m4.xlarge')

----------!

In [127]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_regressor.serializer = csv_serializer
linear_regressor.deserializer = json_deserializer

In [128]:
result = linear_regressor.predict(x_test)
result

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'predictions': [{'score': 53349.7265625},
  {'score': 117360.890625},
  {'score': 126190.015625},
  {'score': 57764.2890625},
  {'score': 44520.6015625},
  {'score': 56660.6484375}]}

In [129]:
predictions = np.array([r['score'] for r in result['predictions']])
predictions

array([ 53349.7265625, 117360.890625 , 126190.015625 ,  57764.2890625,
        44520.6015625,  56660.6484375])

In [131]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': predictions.flatten()})
df

,Actual,Predicted
0,57189.0,53349.726562
1,116969.0,117360.890625
2,122391.0,126190.015625
3,57081.0,57764.289062
4,56642.0,44520.601562
5,56957.0,56660.648438


In [132]:
linear_regressor.delete_endpoint()